# Detector noise

During training, simulated noise $n_I$ is added to waveforms $h_I(\theta)$ measured in detectors to produce realistic simulated data,

$$
d_I = h_I(\theta) + n_I.
$$

Dingo assumes this noise to be stationary and Gaussian, thus it is independent in each frequency bin, with variance given by some power spectral density (PSD).
```{important}
Similar to extrinsic parameters, detector noise is repeatedly sampled **during training** and added to the simulated signal. This augments the training set with new noise realizations for each epoch, reducing overfitting. 
```

Although noise is *mostly* stationary and Guassian during an LVK observing run, the PSD in each detector does tend to drift from event to event. In a usual likelihood-based PE run, this is taken into account by estimating the PSD at the time of the event (either using [Welch's method](https://en.wikipedia.org/wiki/Welch%27s_method) on signal-free data surrounding the event, or at the same time as the event using [BayesWave](https://git.ligo.org/lscsoft/bayeswave)), and using this in the likelihood integral.

Dingo also estimates the PSD just prior to an event and uses this at inference time in two ways:
1. It whitens the data with respect to this PSD.
2. It provides the PSD (or rather, the inverse ASD) as context to the neural network.

A suitably trained model can therefore make use of the PSD as needed to generate the posterior.

(asd-dataset)=
## ASD dataset

To train a model to perform inference conditioned on the noise PSD, it is necessary to not just sample random noise realizations for a given PSD, but also **sample the PSD** from a distribution for a given observing run. Training in this way is necessary to perform fully amortized inference and account for the variation of PSDs from event to event.

The `ASDDataset` class stores a set of ASD samples for several detectors, allowing for sampling during training.

```{eval-rst}
.. autoclass:: dingo.gw.ASD_dataset.noise_dataset.ASDDataset
    :members:
    :inherited-members:
    :show-inheritance:
```

As with the noise realizations, a random ASD is chosen from the dataset when preparing each sample during training. This augments the training set compared to fixing the noise ASD for each sample prior to training.

Similarly to the `WaveformDataset`, the `ASDDataset` is just a container. Dingo includes routines for building such a dataset from observational data. The basic approach is as follows:

1. Identify stretches of data within an observing run meeting certain criteria (sufficiently long, without events, and sufficiently high quality, ...).
2. Download data from [GWOSC](https://www.gw-openscience.org). 
3. Estimate ASDs using Welch's method on these stretches.
4. Save the collection of ASDs.

To generate an ASD dataset, execute the following script:
```bash
dingo_generate_ASD_dataset --data_dir data_dir --settings_file asd_dataset_settings.yaml
```

where the settings file is of the form
```yaml
dataset_settings:
  f_s: 4096
  T_PSD: 1024
  T: 8
  window:
    roll_off: 0.4
    type: tukey
  T_gap: 0
  num_psds_max: 2
  detectors:
    - H1
    - L1
  observing_run: O2
```

Options correspond to the following:

Sampling rate `f_s` (Hz)
: This should be at least twice the value of `f_max` expected to be used.

Data length `T_PSD` (s)
: The entire length of data from which to estimate a PSD using Welch's method. Periodigrams are calculated on segments of this, and then averaged using the `median` method.

Segment length `T` (s)
: The length of each segment on which to take the DFT and calculate a periodigram.

Gap `T_gap` (s)
: Gap between duration-`T` segments. E.g., if `T_PSD=1024`, `T=8`, `T_gap=8`, then for each PSD, 64 periodigrams are computed, each using data stretches 8 s long, with gaps of 8 s between segments. Segments would then be $[0~\text{s}, 8~\text{s}], [16~\text{s}, 24~\text{s}], \ldots$.

Window function
: Parameters of the window function used before taking DFT of data segments.

`num_psds_max` (optional)
: If set, stop building the dataset after this number of PSDs have been estimated. This setting is useful for building a single-PSD dataset for pretraining a network.

Detectors
: Which detectors (H1, L1, V1, ...) to include in the dataset.

Observing run
: Which observing run to use when estimating PSDs.


(ref:window-factor)=
## Data conditioning

Importantly, the variance of *white* noise in each frequency bin is not 1, but rather

$$
\sigma^2_{\text{white}} = \frac{w}{4\delta f}
$$

where $\delta f$ is the frequency resolution and $w$ is a "window factor".

The denominator in the noise variance is seen to arise most easily in the noise-weighted inner product,

$$
(a | b) = 4 \text{Re} \int_{f_\text{min}}^{f_\text{max}} df\, \frac{a^\ast(f)b(f)}{S_{\text{n}}(f)}
$$

The window factor comes in because a window must be applied to time series data prior to taking the FFT. The windowing is assumed to reduce the power in the noise, but not affect the signal (which is localized away from the edge of the data segment). To simulate this, we add noise with variance scaled by the window factor.

The noise standard deviation is stored in the property `FrequencyDomain.noise_std`. The window factor is calculated from the data conditioning settings specified in the train settings file.